In [1]:
import json
import  tensorflow as tf
#from image_transforms import *
import numpy as np
import os
import logging
import pickle

In [2]:
# ouverture du fichiers contenant les paramètres de la calibration des 3 caméras utilisé dans campus
file='calibration_campus.json'
calib_param = open(file)
data = json.load(calib_param)
# 3 cameras : 
camera=data['0']
cam1_param=data['1']
cam2_param=data['2']

In [3]:
# Copie des fonctions codées dans 'image_transforms.py ' pour les modifier directement ici

@tf.function
def tf_get_cam_params(camera):
    R = tf.constant(camera['R'], dtype=tf.float32)
    T = tf.constant(camera['T'], dtype=tf.float32)
    fx = tf.constant(camera['fx'], dtype=tf.float32)
    fy = tf.constant(camera['fy'], dtype=tf.float32)
    f = tf.stack([fx, fy], axis=-1)#.reshape(-1,2,1) 
    #tf.reshape(f,2,1) ???
    cx = tf.constant(camera['cx'], dtype=tf.float32)
    cy = tf.constant(camera['cy'], dtype=tf.float32)
    c = tf.stack([cx, cy], axis=-1)#.reshape(-1,2,1)
    k = tf.constant(camera['k'], dtype=tf.float32)
    p = tf.constant(camera['p'], dtype=tf.float32)
    return R, T, f, c, k, p





@tf.function
def tf_point_3d_to_2d(points, R, T, f, c, k, p):
    ####
    # quelques petit ajustements pour la base campus 
    Np = 1
    Nj = 17
    # histoire de tester ....
    B=1
    Nd=3
 #   B, Np, Nj, Nd = points.shape # [batch_size, Number of persons, Number of joints, Dimension]
    # points = points.reshape(B, -1, 3).transpose(1, 2) # [B, Np*Nj, Nd] --> [B, Nd, Np*Nj]
    points_cam = tf.matmul(R, points - T) # Translation + Rotation
    out = points_cam[:, :2, :]/(points_cam[:, 2:, :] + 1e-5) # devision by depth (and avoiding division by 0)
    
    # Camera distortion
    r = tf.reduce_sum(out ** 2, axis=1) # [B, Np*Nj]
    d = 1 + k[:, 0] * r + k[:, 1] * (r ** 2) + k[:, 2] * (r ** 3)
    u = out[:, 0, :] * d + 2 * p[:, 0] * out[:, 0, :] * out[:, 1, :] + p[:, 1] * (r + 2 * out[:, 0, :] * out[:, 0, :])  # [B, Np*Nj]
    v = out[:, 1, :] * d + 2 * p[:, 1] * out[:, 0, :] * out[:, 1, :] + p[:, 0] * (r + 2 * out[:, 1, :] * out[:, 1, :])  # [B, Np*Nj]
    out = tf.stack([u, v], axis=1)
    out_pixel = f * out + c # [B, 2, Np*Nj]
    out_pixel = out_pixel.transpose(1, 2) # [B, Np*Nj, 2]
    
    # get back the original shape
    out_pixel = out_pixel.reshape(B, Np, Nj, 2) # [B, Np, Nj, 2]
    return out_pixel

In [4]:
# Pour la camera 0 par exemple
# Test de la fonction 1 
[R, T, f, c, k, p] = tf_get_cam_params(camera)

In [5]:
# ouverture des données 
def open_data():
        fp = os.path.join( "pred_campus_maskrcnn_hrnet_coco.pkl")
        with open(fp, "rb") as f:
            logging.info("=> load {}".format(fp))
            pred = pickle.load(f)

        return pred
    
data_campus=open_data()
data_campus

defaultdict(list,
            {'0_471': [{'pred': array([[112.75502777,  90.83792877,   0.71590805],
                      [110.36601257,  88.44891357,   0.58689582],
                      [112.15777588,  89.04616547,   0.78871667],
                      [105.58797455,  89.04616547,   0.88350916],
                      [111.56051636,  89.64341736,   0.91358453],
                      [102.00444794, 100.39399719,   0.90975225],
                      [114.54679108,  99.7967453 ,   0.90894276],
                      [100.80994415, 114.72809601,   0.83392549],
                      [115.14404297, 115.32535553,   0.87729979],
                      [102.00444794, 124.88142395,   0.7768299 ],
                      [116.93580627, 127.86769104,   0.88129747],
                      [103.79621124, 127.27043915,   0.81829   ],
                      [111.56051636, 127.27043915,   0.84221137],
                      [109.17150116, 148.17433167,   0.79489326],
                      [110.36601257, 148.

In [7]:
# on choisit un point au hasard parmi les données 
points = data_campus['0_104']
points = points[0]['pred']
points

array([[ 96.66855621, 100.09867859,   0.69933951],
       [ 94.35126495,  96.6227417 ,   0.60400379],
       [ 97.82720184,  96.6227417 ,   0.83063871],
       [ 82.7647934 ,  98.94003296,   0.95099425],
       [ 96.66855621,  98.94003296,   0.96974158],
       [ 76.97154999, 119.79568481,   0.92044753],
       [ 98.9858551 , 119.79568481,   0.92628658],
       [ 71.17831421, 147.60322571,   0.92528492],
       [105.93773651, 147.60322571,   0.89967388],
       [ 72.33696747, 170.77616882,   0.92769563],
       [108.25502777, 170.77616882,   0.92813522],
       [ 79.28884888, 170.77616882,   0.83192194],
       [ 95.50991058, 170.77616882,   0.84152555],
       [ 88.55802917, 205.53559875,   0.8716495 ],
       [100.14450073, 206.69424438,   0.91385782],
       [ 95.50991058, 239.1363678 ,   0.90506005],
       [ 85.08208466, 242.61230469,   0.88194442]])

In [8]:
tensor_pts = tf.convert_to_tensor(points, dtype=tf.float32)
tensor_pts = tf.reshape(tensor_pts,(3,17))
tensor_pts

<tf.Tensor: shape=(3, 17), dtype=float32, numpy=
array([[ 96.66856   , 100.09868   ,   0.6993395 ,  94.351265  ,
         96.62274   ,   0.6040038 ,  97.8272    ,  96.62274   ,
          0.8306387 ,  82.76479   ,  98.94003   ,   0.95099425,
         96.66856   ,  98.94003   ,   0.9697416 ,  76.97155   ,
        119.795685  ],
       [  0.9204475 ,  98.985855  , 119.795685  ,   0.9262866 ,
         71.178314  , 147.60323   ,   0.9252849 , 105.93774   ,
        147.60323   ,   0.8996739 ,  72.33697   , 170.77617   ,
          0.92769563, 108.25503   , 170.77617   ,   0.9281352 ,
         79.28885   ],
       [170.77617   ,   0.83192194,  95.50991   , 170.77617   ,
          0.84152555,  88.55803   , 205.5356    ,   0.8716495 ,
        100.1445    , 206.69424   ,   0.9138578 ,  95.50991   ,
        239.13637   ,   0.90506005,  85.082085  , 242.6123    ,
          0.8819444 ]], dtype=float32)>

In [9]:
# test de la fonction 2
tf_point_3d_to_2d(tensor_pts, R, T, f, c, k, p)

ValueError: in user code:

    <ipython-input-3-e64229470671>:34 tf_point_3d_to_2d  *
        out = points_cam[:, :2, :]/(points_cam[:, 2:, :] + 1e-5) # devision by depth (and avoiding division by 0)
    /home/spi-2017-12/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /home/spi-2017-12/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1041 _slice_helper
        return strided_slice(
    /home/spi-2017-12/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /home/spi-2017-12/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1214 strided_slice
        op = gen_array_ops.strided_slice(
    /home/spi-2017-12/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/gen_array_ops.py:10538 strided_slice
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /home/spi-2017-12/ana